# Классификация данных методом *k* ближайших соседей

Работу выполнили студенты группы БСМО-11-25:

* Аршинов Владислав
* Елохин Валерий
* Савин Даниил
* Якушенко Иван



Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter

Загружаем собранные данные

In [ ]:
data = pd.read_csv('Датасет CSV.csv', sep=';', encoding='utf-8')

Разделяем данные на признаки (`Х`) и целевую переменную (`у`)

In [ ]:
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

Разделяем данные на обучающую и тестовую выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Масштабируем признаки для того, чтобы признаки с большими числовыми значениями не доминировали над признаками с меньшими значениями

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Реализуем метод *k*-ближайших соседей:
1. Вычисляем Евклидовы расстояния от нового объекта до каждого объекта обучающей выборки
2. Сортируем по расстоянию по возрастанию и выбираем *k* ближайших соседей
3. С помощью голосования большинством прогнозируем класс нового объекта (Голосование большиством - это метод определения итогового класса объекта на основе классов его *k* ближайших соседей)

In [ ]:
def knn_predict(X_train, y_train, X_test, k=5):
    predictions = []
    for test_point in X_test:
        distances = []
        for i, train_point in enumerate(X_train):
            dist = np.sqrt(np.sum((test_point - train_point) ** 2))
            distances.append((dist, y_train[i]))

        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:k]
        # Голосование большинством
        k_labels = [label for _, label in k_nearest]
        most_common = Counter(k_labels).most_common(1)[0][0]
        predictions.append(most_common)

    return predictions

Проводим оценку точности и ищем наилучшее *k*

In [ ]:
max_accuracy = 0
max_accuracy_k = 0
for k in range (1,20):
    y_pred = knn_predict(X_train, y_train, X_test, k)
    accuracy = np.sum(y_pred == y_test) / len(y_test)
    if accuracy > max_accuracy:
      max_accuracy = accuracy
      max_accuracy_k = k
    print(f"{k}:\tТочность модели: {accuracy:.2f}")

print(f"\n\nМаксимальная точность {max_accuracy:.2f} достигается при k = {max_accuracy_k}")

1:	Точность модели: 0.60
2:	Точность модели: 0.60
3:	Точность модели: 0.50
4:	Точность модели: 0.60
5:	Точность модели: 0.50
6:	Точность модели: 0.60
7:	Точность модели: 0.50
8:	Точность модели: 0.70
9:	Точность модели: 0.70
10:	Точность модели: 0.70
11:	Точность модели: 0.60
12:	Точность модели: 0.70
13:	Точность модели: 0.60
14:	Точность модели: 0.70
15:	Точность модели: 0.60
16:	Точность модели: 0.50
17:	Точность модели: 0.60
18:	Точность модели: 0.50
19:	Точность модели: 0.60


Максимальная точность 0.70 достигается при k = 8


Предсказываем результат для найденного наилучшего *k* и выводим его на экран

In [ ]:
y_pred = knn_predict(X_train, y_train, X_test, max_accuracy_k)
print(f"\nПримеры предсказаний (первые 10):")
print("Фактически\t|\tПредсказано")
print("----------\t|\t-----------")
for i in range(min(10, len(y_test))):
    actual = "Айфон" if y_test[i] == 1 else "Андроид"
    predicted = "Айфон" if y_pred[i] == 1 else "Андроид"
    print(f"{actual}   \t|\t{predicted}")


Примеры предсказаний (первые 10):
Фактически	|	Предсказано
----------	|	-----------
Андроид   	|	Айфон
Айфон   	|	Айфон
Айфон   	|	Айфон
Андроид   	|	Андроид
Андроид   	|	Андроид
Айфон   	|	Андроид
Андроид   	|	Айфон
Андроид   	|	Андроид
Айфон   	|	Айфон
Айфон   	|	Айфон
